In [44]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy

In [45]:
from ray.rllib.agents.ppo.ppo_torch_policy import PPOTorchPolicy
from ray.rllib.agents.a3c.a3c_torch_policy import A3CTorchPolicy
from ray.rllib.agents.a3c.a2c import A2CTrainer

In [46]:
import gym
from typing import Optional, Dict

import ray
from ray.rllib.agents.ppo.ppo_torch_policy import ValueNetworkMixin
from ray.rllib.evaluation.episode import MultiAgentEpisode
from ray.rllib.evaluation.postprocessing import compute_gae_for_sample_batch, \
    Postprocessing
from ray.rllib.models.action_dist import ActionDistribution
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.policy.policy import Policy
from ray.rllib.policy.policy_template import build_policy_class
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.utils.annotations import Deprecated
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.torch_ops import apply_grad_clipping, sequence_mask
from ray.rllib.utils.typing import TrainerConfigDict, TensorType, \
    PolicyID, LocalOptimizer

torch, nn = try_import_torch()

In [47]:
def after_init(policy: Policy, obs_space: gym.spaces.Space, 
              action_space: gym.spaces.Space, config: TrainerConfigDict)->None:
        policy.past_len = 5        
        policy.past_models = deque(maxlen =policy.past_len)
        policy.timestep = 0
    

In [48]:
def compute_div_loss(policy: Policy, model: ModelV2,
                      dist_class: ActionDistribution,
                      train_batch: SampleBatch):
    logits, _ = model.from_batch(train_batch)
    values = model.value_function()
    valid_mask = torch.ones_like(values, dtype=torch.bool)
    dist = dist_class(logits, model)
    log_probs = dist.logp(train_batch[SampleBatch.ACTIONS]).reshape(-1)
    
    divs = []
    for idx, past_model in enumerate(policy.past_models):
        logits, _ = past_model.from_batch(train_batch)
        values = past_model.value_function()
        valid_mask = torch.ones_like(values, dtype=torch.bool)
        dist = dist_class(logits, past_model)
        past_log_probs = dist.logp(train_batch[SampleBatch.ACTIONS]).reshape(-1) 
        div = div_metric(log_probs, past_log_probs).sum(1)
        div = div.mean(0)
        divs.append(div)
    
    divs_sort_idx = np.argsort([d.data[0] for d in divs])
    div_loss_orig = 0
    for idx in divs_sort_idx:
        div_loss += divs[idx]
        div_loss_orig += divs[idx]
    
    div_loss = div_loss / self.past_len
    
    return div_loss

In [49]:
def actor_critic_loss(policy: Policy, model: ModelV2,
                      dist_class: ActionDistribution,
                      train_batch: SampleBatch) -> TensorType:
    logits, _ = model.from_batch(train_batch)
    values = model.value_function()
    policy.timestep += 1
    
    if policy.timestep % 100 == 0:
        policy.past_models.append(copy.deepcopy(model))
    
    if policy.is_recurrent():
        B = len(train_batch[SampleBatch.SEQ_LENS])
        max_seq_len = logits.shape[0] // B
        mask_orig = sequence_mask(train_batch[SampleBatch.SEQ_LENS],
                                  max_seq_len)
        valid_mask = torch.reshape(mask_orig, [-1])
    else:
        valid_mask = torch.ones_like(values, dtype=torch.bool)

    dist = dist_class(logits, model)
    log_probs = dist.logp(train_batch[SampleBatch.ACTIONS]).reshape(-1)
    pi_err = -torch.sum(
        torch.masked_select(log_probs * train_batch[Postprocessing.ADVANTAGES],
                            valid_mask))

    # Compute a value function loss.
    if policy.config["use_critic"]:
        value_err = 0.5 * torch.sum(
            torch.pow(
                torch.masked_select(
                    values.reshape(-1) -
                    train_batch[Postprocessing.VALUE_TARGETS], valid_mask),
                2.0))
    # Ignore the value function.
    else:
        value_err = 0.0

    entropy = torch.sum(torch.masked_select(dist.entropy(), valid_mask))


    total_loss = (pi_err + value_err * policy.config["vf_loss_coeff"] -
                  entropy * policy.config["entropy_coeff"] - compute_div_loss(policy, model, dist_class, train_batch))

    policy.entropy = entropy
    policy.pi_err = pi_err
    policy.value_err = value_err

    return total_loss

In [50]:
CustomPolicy = A3CTorchPolicy.with_updates(
    name="MyCustomA3CTorchPolicy",
    loss_fn=actor_critic_loss,
    after_init=after_init)
CustomTrainer = A2CTrainer.with_updates(
    default_policy=CustomPolicy)

In [51]:
tune.run(CustomTrainer, config={"env": 'Frostbite-v0', "num_gpus":1})

Trial name,status,loc
A2C_Frostbite-v0_74b39_00000,PENDING,


(pid=2409) 2021-12-19 15:15:51,512	INFO trainer.py:741 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=2409) 2021-12-19 15:15:51,512	INFO trainer.py:758 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
A2C_Frostbite-v0_74b39_00000,RUNNING,


Trial name,status,loc
A2C_Frostbite-v0_74b39_00000,RUNNING,


(pid=2409) 2021-12-19 15:15:59,553	WARNING trainer_template.py:185 -- `execution_plan` functions should accept `trainer`, `workers`, and `config` as args!
(pid=2409) 2021-12-19 15:15:59,554	WARNING util.py:57 -- Install gputil for GPU system monitoring.


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
A2C_Frostbite-v0_74b39_00000,RUNNING,132.236.59.202:2409,50,503.154,368400,134.6,260,10,580.52


2021-12-19 15:24:31,506	WARNING tune.py:574 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
A2C_Frostbite-v0_74b39_00000,RUNNING,132.236.59.202:2409,50,503.154,368400,134.6,260,10,580.52


(pid=2444) 2021-12-19 15:24:31,519	ERROR worker.py:428 -- SystemExit was raised from the worker
(pid=2444) Traceback (most recent call last):
(pid=2444)   File "python/ray/_raylet.pyx", line 684, in ray._raylet.task_execution_handler
(pid=2444)   File "python/ray/_raylet.pyx", line 524, in ray._raylet.execute_task
(pid=2444)   File "python/ray/_raylet.pyx", line 561, in ray._raylet.execute_task
(pid=2444)   File "python/ray/_raylet.pyx", line 568, in ray._raylet.execute_task
(pid=2444)   File "python/ray/_raylet.pyx", line 572, in ray._raylet.execute_task
(pid=2444)   File "python/ray/_raylet.pyx", line 522, in ray._raylet.execute_task.function_executor
(pid=2444)   File "/home/ml2558/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/ray/_private/function_manager.py", line 579, in actor_method_executor
(pid=2444)     return method(__ray_actor, *args, **kwargs)
(pid=2444)   File "/home/ml2558/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/ray/util/tracing/tracing_helper.py", line 4

2021-12-19 15:24:31,720	ERROR tune.py:613 -- Trials did not complete: [A2C_Frostbite-v0_74b39_00000]
2021-12-19 15:24:31,721	INFO tune.py:617 -- Total run time: 523.03 seconds (522.78 seconds for the tuning loop).
2021-12-19 15:24:31,721	WARNING tune.py:621 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
